In [1]:

%run ../../load_magic/storage.py
%run ../../load_magic/dataframes.py
%pprint
import re
from math import cos, sin, pi
import numpy as np
%who

DATA_FOLDER: ../data/
SAVES_FOLDER: ../saves/
Pretty printing has been turned OFF
DATA_CSV_FOLDER	 DATA_FOLDER	 ENCODING_TYPE	 SAVES_CSV_FOLDER	 SAVES_FOLDER	 SAVES_PICKLE_FOLDER	 attempt_to_pickle	 cos	 example_iterrows	 
get_column_descriptions	 load_csv	 load_dataframes	 load_object	 math	 np	 os	 pd	 pi	 
pickle	 re	 save_dataframes	 sin	 store_objects	 sys	 


In [2]:

dataframes_dict = load_dataframes(generations_df='generations_df', patriline_df='patriline_df',
                                  turnings_df='turnings_df')

Attempting to load C:\Users\dev\Documents\repositories\notebooks\Strauss–Howe\saves\pickle\generations_df.pickle.
No pickle exists - attempting to load C:\Users\dev\Documents\repositories\notebooks\Strauss–Howe\saves\csv\generations_df.csv.
Attempting to load C:\Users\dev\Documents\repositories\notebooks\Strauss–Howe\saves\pickle\patriline_df.pickle.
No pickle exists - attempting to load C:\Users\dev\Documents\repositories\notebooks\Strauss–Howe\saves\csv\patriline_df.csv.
Attempting to load C:\Users\dev\Documents\repositories\notebooks\Strauss–Howe\saves\pickle\turnings_df.pickle.
No pickle exists - attempting to load C:\Users\dev\Documents\repositories\notebooks\Strauss–Howe\saves\csv\turnings_df.csv.


In [3]:

index_name = 'Patriarch Name'
patriline_df = dataframes_dict['patriline_df']
if 'Unnamed: 0' in patriline_df.columns:
    patriline_df = patriline_df.set_index('Unnamed: 0')
elif index_name in patriline_df.columns:
    patriline_df = patriline_df.set_index(index_name)
patriline_df.index.name = index_name

In [4]:

index_name = 'generation_name'
generations_df = dataframes_dict['generations_df']
if 'Unnamed: 0' in generations_df.columns:
    generations_df = generations_df.set_index('Unnamed: 0')
elif index_name in generations_df.columns:
    generations_df = generations_df.set_index(index_name)
generations_df.index.name = index_name

In [5]:

index_name = 'turning_name'
turnings_df = dataframes_dict['turnings_df']
if 'Unnamed: 0' in turnings_df.columns:
    turnings_df = turnings_df.set_index('Unnamed: 0')
elif index_name in turnings_df.columns:
    turnings_df = turnings_df.set_index(index_name)
turnings_df.index.name = index_name

In [6]:

# Remove Late Medeival Saeculum
mask_series = (turnings_df['saeculum_name'] == 'Late Medeival')
turnings_df = turnings_df[~mask_series]

In [7]:

# Make balanced saeculum list
turnings_group = turnings_df.groupby(by='saeculum_name', axis='index')
saeculum_list = []
for key_str, group_df in turnings_group:
    saeculum_list.append({key_str: group_df['turning_begin_year'].tolist()})
saeculum_list.sort(key=lambda x: list(x.values())[0][0], reverse=False)
saeculum_list

[{'Tudor': [1487, 1517, 1542, 1569]}, {'New World': [1594, 1621, 1649, 1675]}, {'Revolutionary': [1704, 1727, 1746, 1773]}, {'Civil War': [1794, 1822, 1844, 1860]}, {'Great Power': [1865, 1886, 1908, 1929]}, {'Millennial': [1946, 1964, 1984, 2008]}]

In [8]:

# Interpolate missing radii
interpolated_radius_list = []
saeculum_radius = 0
for saeculum_dict in saeculum_list:
    saeculum_radius += 100
    interpolated_radius_list.append(saeculum_radius)
    for saeculum_name, year_list in saeculum_dict.items():
        for i, year in enumerate(year_list):
            if i > 0:
                interpolated_radius_list.append(pd.np.nan)
interpolated_radius_list = interpolated_radius_list + [len(saeculum_list)*100+100]
interpolated_radius_list = pd.Series(interpolated_radius_list).interpolate().tolist()
print(len(interpolated_radius_list), interpolated_radius_list)

25 [100.0, 125.0, 150.0, 175.0, 200.0, 225.0, 250.0, 275.0, 300.0, 325.0, 350.0, 375.0, 400.0, 425.0, 450.0, 475.0, 500.0, 525.0, 550.0, 575.0, 600.0, 625.0, 650.0, 675.0, 700.0]


In [9]:

begin_year_list = sorted(turnings_df['turning_begin_year'])
begin_year_list += [turnings_df['turning_end_year'].max()+1]
print(len(begin_year_list), begin_year_list)

25 [1487, 1517, 1542, 1569, 1594, 1621, 1649, 1675, 1704, 1727, 1746, 1773, 1794, 1822, 1844, 1860, 1865, 1886, 1908, 1929, 1946, 1964, 1984, 2008, 2030]


In [10]:

ccw_theta_list = [270 - i*90 for i in range(len(begin_year_list))]
print(len(ccw_theta_list), ccw_theta_list)

25 [270, 180, 90, 0, -90, -180, -270, -360, -450, -540, -630, -720, -810, -900, -990, -1080, -1170, -1260, -1350, -1440, -1530, -1620, -1710, -1800, -1890]


In [11]:

cw_theta_list = [270 + i*90 for i in range(len(begin_year_list))]
print(len(cw_theta_list), cw_theta_list)

25 [270, 360, 450, 540, 630, 720, 810, 900, 990, 1080, 1170, 1260, 1350, 1440, 1530, 1620, 1710, 1800, 1890, 1980, 2070, 2160, 2250, 2340, 2430]


In [12]:

year_tuple_list = [(y1, y2) for y1, y2 in zip(begin_year_list[:-1],
                                              begin_year_list[1:])]
radius_tuple_list = [(r1, r2) for r1, r2 in zip(interpolated_radius_list[:-1],
                                                interpolated_radius_list[1:])]
cw_theta_tuple_list = [(t1, t2) for t1, t2 in zip(cw_theta_list[:-1],
                                                  cw_theta_list[1:])]
ccw_theta_tuple_list = [(t1, t2) for t1, t2 in zip(ccw_theta_list[:-1],
                                                   ccw_theta_list[1:])]

In [13]:

# Create counter-clockwse dictionary
ccw_history_radius_dict = {}
ccw_history_year_dict = {}
ccw_history_theta_dict = {}
for year, radius, theta in zip(begin_year_list, interpolated_radius_list, ccw_theta_list):
    ccw_history_radius_dict[int(radius)] = (year, theta)
    ccw_history_year_dict[int(year)] = (radius, theta)
    ccw_history_theta_dict[int(theta)] = (year, radius)
for year_tuple, radius_tuple, theta_tuple in zip(year_tuple_list, radius_tuple_list,
                                                 ccw_theta_tuple_list):
    start_radius, stop_radius = radius_tuple
    increment_count = int(2*pi*start_radius)
    radius_array = np.linspace(start=start_radius, stop=stop_radius,
                               num=increment_count)
    start_year, stop_year = year_tuple
    year_array = np.linspace(start=start_year, stop=stop_year,
                             num=increment_count)
    start_theta, stop_theta = theta_tuple
    theta_array = np.linspace(start=start_theta, stop=stop_theta,
                              num=increment_count)
    for radius, theta, year in zip(radius_array, theta_array, year_array):
        if int(radius) not in ccw_history_radius_dict:
            ccw_history_radius_dict[int(radius)] = (year, theta)
        if int(year) not in ccw_history_year_dict:
            ccw_history_year_dict[int(year)] = (radius, theta)
        if int(theta) not in ccw_history_theta_dict:
            ccw_history_theta_dict[int(theta)] = (year, radius)
store_objects(ccw_history_radius_dict=ccw_history_radius_dict,
              ccw_history_year_dict=ccw_history_year_dict,
              ccw_history_theta_dict=ccw_history_theta_dict)

Pickling to C:\Users\dev\Documents\repositories\notebooks\Strauss–Howe\saves\pickle\ccw_history_radius_dict.pickle
Pickling to C:\Users\dev\Documents\repositories\notebooks\Strauss–Howe\saves\pickle\ccw_history_year_dict.pickle
Pickling to C:\Users\dev\Documents\repositories\notebooks\Strauss–Howe\saves\pickle\ccw_history_theta_dict.pickle


In [14]:

# Create clockwse dictionary
cw_history_radius_dict = {}
cw_history_year_dict = {}
cw_history_theta_dict = {}
for year, radius, theta in zip(begin_year_list, interpolated_radius_list, cw_theta_list):
    cw_history_radius_dict[int(radius)] = (year, theta)
    cw_history_year_dict[int(year)] = (radius, theta)
    cw_history_theta_dict[int(theta)] = (year, radius)
for year_tuple, radius_tuple, theta_tuple in zip(year_tuple_list, radius_tuple_list,
                                                 cw_theta_tuple_list):
    start_radius, stop_radius = radius_tuple
    increment_count = int(2*pi*start_radius)
    radius_array = np.linspace(start=start_radius, stop=stop_radius,
                               num=increment_count)
    start_year, stop_year = year_tuple
    year_array = np.linspace(start=start_year, stop=stop_year,
                             num=increment_count)
    start_theta, stop_theta = theta_tuple
    theta_array = np.linspace(start=start_theta, stop=stop_theta,
                              num=increment_count)
    for radius, theta, year in zip(radius_array, theta_array, year_array):
        if int(radius) not in cw_history_radius_dict:
            cw_history_radius_dict[int(radius)] = (year, theta)
        if int(year) not in cw_history_year_dict:
            cw_history_year_dict[int(year)] = (radius, theta)
        if int(theta) not in cw_history_theta_dict:
            cw_history_theta_dict[int(theta)] = (year, radius)
store_objects(cw_history_radius_dict=cw_history_radius_dict,
              cw_history_year_dict=cw_history_year_dict,
              cw_history_theta_dict=cw_history_theta_dict)

Pickling to C:\Users\dev\Documents\repositories\notebooks\Strauss–Howe\saves\pickle\cw_history_radius_dict.pickle
Pickling to C:\Users\dev\Documents\repositories\notebooks\Strauss–Howe\saves\pickle\cw_history_year_dict.pickle
Pickling to C:\Users\dev\Documents\repositories\notebooks\Strauss–Howe\saves\pickle\cw_history_theta_dict.pickle


In [15]:
%%javascript
var kernel = IPython.notebook.kernel;
var body = document.body,  
    attribs = body.attributes;
var command = "notebook_path = " + "'"+attribs['data-notebook-path'].value+"'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [16]:
# %load ../../load_magic/nbviewer.py

from IPython.display import HTML

notebook_viewer_url = 'https://nbviewer.jupyter.org/github/dbabbitt/notebooks/blob/master/'
notebook_viewer_url += '/'.join(notebook_path.split('/')[1:])
html_str = 'Click <a href="{}" target="_blank">here</a> to view notebook in nbviewer.'
HTML(html_str.format(notebook_viewer_url))